In [1]:
# import required packages
import cv2
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import pickle

# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all train images
train_generator = train_data_gen.flow_from_directory(
    'data/train',
    target_size=(48, 48),
    batch_size=64,
    color_mode="rgb",  # Change to 'rgb' for RGB images
    class_mode='categorical')

# Preprocess all test images
validation_generator = validation_data_gen.flow_from_directory(
    'data/test',
    target_size=(48, 48),
    batch_size=64,
    color_mode="rgb",  # Change to 'rgb' for RGB images
    class_mode='categorical')

# Load the pre-trained MobileNetV2 model without the top classification layers
mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the pre-trained layers
for layer in mobilenet_model.layers:
    layer.trainable = False

# Add custom classification layers
x = mobilenet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(7, activation='softmax')(x)

# Create the final model
emotion_model = Model(inputs=mobilenet_model.input, outputs=x)

# Compile the model
optimizer = Adam(learning_rate=0.0001)
emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=7178 // 64)

# Save the model structure and weights
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

emotion_model.save_weights('emotion_model.h5')

# Save the history object
with open('emotion_model_history.pkl', 'wb') as f:
    pickle.dump(emotion_model_info.history, f)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


9406464/9406464 [==============================] - 1s 0us/step


C:\Users\JJ\AppData\Local\Temp\ipykernel_17232\2454714967.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/50


448/448 [==============================] - 36s 69ms/step - loss: 1.8384 - accuracy: 0.2987 - val_loss: 1.6018 - val_accuracy: 0.3728
Epoch 2/50
448/448 [==============================] - 38s 85ms/step - loss: 1.6123 - accuracy: 0.3686 - val_loss: 1.5691 - val_accuracy: 0.3912
Epoch 3/50
448/448 [==============================] - 53s 118ms/step - loss: 1.5291 - accuracy: 0.4111 - val_loss: 1.5447 - val_accuracy: 0.3977
Epoch 4/50
448/448 [==============================] - 54s 120ms/step - loss: 1.4662 - accuracy: 0.4387 - val_loss: 1.5273 - val_accuracy: 0.4085
Epoch 5/50
448/448 [==============================] - 52s 116ms/step - loss: 1.4061 - accuracy: 0.4642 - val_loss: 1.5133 - val_accuracy: 0.4164
Epoch 6/50
448/448 [==============================] - 53s 118ms/step - loss: 1.3608 - accuracy: 0.4876 - val_loss: 1.5037 - val_accuracy: 0.4254
Epoch 7/50
448/448 [==============================] - 53s 119ms/step - loss: 1.3098 - accuracy: 0.5135 - val_loss: 1.4955 - val_ac